<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week15.do.txt --no_mako --no_abort -->
<!-- dom:TITLE: Week 18 May 1-5: Neural networks and project 2 -->

# Week 18 May 1-5: Neural networks and project 2
**Morten Hjorth-Jensen  Email morten.hjorth-jensen@fys.uio.no**, Department of Physics and Center fo Computing in Science Education, University of Oslo, Oslo, Norway and Department of Physics and Astronomy and Facility for Rare Ion Beams, Michigan State University, East Lansing, Michigan, USA

Date: **May 3, 2023**

Copyright 1999-2023, Morten Hjorth-Jensen  Email morten.hjorth-jensen@fys.uio.no. Released under CC Attribution-NonCommercial 4.0 license

## Overview of week 18, May 1-5
* Neural networks and project 2

a. Discussions of neural network and solution of project 2

b. Discussion of codes